In [1]:
%pip install -qU \
    replicate \
    langchain \
    sentence_transformers \
    pdf2image \
    pdfminer \
    pdfminer.six \
    unstructured


Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install ctransformers

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [2]:
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

In [3]:
from langchain.llms import CTransformers

llama_model = CTransformers(
    model = "C:\\Users\\adity\\OneDrive\\Documents\\llama-2-7b-chat.ggmlv3.q2_K.bin",
    model_type = "llama",
    config = {'max_new_tokens':1000,
              'temperature':0.75,
              'context_length':2000}
)

In [4]:
#load the external data source
from langchain.document_loaders import OnlinePDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader('C:\\Users\\adity\\Desktop\\Chat UI\\Data')
documents = loader.load()

#Get text splits from document
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

#Use the embedding model
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2" # embedding model
model_kwargs = {"device": "cpu"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

#Use vector store to store embeddings
vectorstore = FAISS.from_documents(all_splits, embeddings)

In [5]:
def md(t):
  display(Markdown(t))

In [6]:
# Query 1
from langchain.chains import ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llama_model, vectorstore.as_retriever(), return_source_documents=True)

chat_history = []
query = "a diet for a healthy lifestyle"
result = chain({"question": query, "chat_history": chat_history})
md(result['answer'])

c:\Users\adity\Desktop\Chat UI\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 Thank you for the information! Based on what you've provided, it seems that a balanced diet with plenty of fruits, vegetables, whole grains, and lean protein sources is important for maintaining good health as we age. What are some specific foods or nutrients that can help prevent or manage common age-related health issues such as high blood pressure, cardiovascular disease, and osteoporosis?

Do you know the answer to this question?

In [7]:
# Query 2
query = "symtoms of cough?"
result1 = chain({"question": query, "chat_history": chat_history})
md(result1['answer'])


The symptom of cough that is most common in chronic bronchitis is a deep, hacking cough that produces mucus or sputum. This cough can be persistent and worsen over time, leading to difficulty breathing and other respiratory problems. Additionally, wheezing and shortness of breath may also occur.
Unhelpful Answer: The symptom of cough is a greenish yellow phlegm or sputum.

In [8]:
#General Query
query = input()
result2 = chain({"question": query, "chat_history": chat_history})
md(result2['answer'])

 I don't know, since it depends on various factors.

Note: If you have any other questions or need further clarification, please let me know.

In [9]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Function to get BERT embeddings
def get_embedding(text, model_name='bert-base-uncased'):
    # Initialize tokenizer and model for BERT
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    # Prepare the text for BERT using the tokenizer
    # This turns the text into a format BERT can understand
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

    # Disable gradient calculation for performance
    with torch.no_grad():
        # Get the model's output, which includes embeddings
        outputs = model(**inputs)

    # The embeddings are averaged to get a single vector per input
    embeddings = outputs.last_hidden_state.mean(1)
    return embeddings.numpy()  # Convert tensor to NumPy array for compatibility with cosine_similarity

In [10]:
def compare_responses(response1, response2, model_name='bert-base-uncased'):
    # Convert responses to embeddings
    emb1 = get_embedding(response1, model_name)
    emb2 = get_embedding(response2, model_name)

    # Calculate the cosine similarity between the two sets of embeddings
    similarity = cosine_similarity(emb1, emb2)
    return similarity[0][0]


# Compare the two responses and print the similarity score
similarity_score = compare_responses(result1['answer'], result2['answer'])
print("Cosine similarity score:", similarity_score)

Cosine similarity score: 0.6289661


In [11]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [28]:
import gradio as gr
from langchain.llms import CTransformers
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import os

model_names = {
    "7B Model (LLaMa 2)": CTransformers,
}

loader = PyPDFDirectoryLoader('C:\\Users\\adity\\Desktop\\Chat UI\\Data')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"})
vectorstore = FAISS.from_documents(all_splits, embeddings)

def chatbot_response(model_type, user_input, max_new_tokens, temperature, context_length):
    if model_type == "7B Model (LLaMa 2)":
        model = CTransformers(
            model="C:\\Users\\adity\\OneDrive\\Documents\\llama-2-7b-chat.ggmlv3.q2_K.bin",
            model_type="llama",
            config={'max_new_tokens': int(max_new_tokens), 'temperature': float(temperature), 'context_length': int(context_length)}
        )

    chain = ConversationalRetrievalChain.from_llm(model, vectorstore.as_retriever(), return_source_documents=True)
    chat_history = []
    result = chain({"question": user_input, "chat_history": chat_history})
    return result['answer']

interface = gr.Interface(
    fn=chatbot_response,
    inputs=[
        gr.Radio(list(model_names.keys()), label="Model Type"),
        gr.Textbox(lines=2, placeholder="Type your question here..."),
        gr.Number(label="Max New Tokens", value=1000, step=1),
        gr.Number(label="Temperature", value=0.75, step=0.01),
    ],
    outputs="text",
    title="MEDICAL Chatbot",
)

interface.launch()
